# Explore here

In [13]:
import pandas as pd

total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv")
total_data.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [14]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [15]:
duplicates = total_data.duplicated().sum()
print(duplicates)

630


In [18]:
total_data2 = total_data.drop_duplicates()
total_data2 = total_data2.reset_index(inplace = False, drop = True)
print("Shape de los Datos", total_data2.shape)
print(f"Spam: {len(total_data2.loc[total_data2.is_spam == 1])}")
print(f"No spam: {len(total_data2.loc[total_data2.is_spam == 0])}")

Shape de los Datos (2369, 2)
Spam: 244
No spam: 2125


In [ ]:
import re


def preprocess_url(url):
    # Convertir a minúsculas
    url = url.lower()

    # Eliminar todos los signos de puntuación y los términos comunes
    url = re.sub(r'[^\w\s]|www|https?|http|com', '', url)

    url = re.sub(r'\d+', '', url)

    # Dividir la URL en partes
 # Encontrar y seleccionar solo palabras completas
    words = re.findall(r'\b[a-zA-Z]+\b', url)

    # Formatear las palabras entre corchetes []
    url_processed = "[" + ",".join(words) + "]"

    return url_processed

# Aplicar el preprocesamiento a la columna "URL" del DataFrame
df["url"] = df["url"].apply(preprocess_url)

# Mostrar las primeras filas del DataFrame para verificar el resultado
df

(2369, 2)
Spam: 244
No spam: 2125


In [ ]:
import re
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Función para extraer tokens de una URL procesada
def extract_tokens(url):
    # Remover corchetes y comas
    url = re.sub(r'[\[\],]', '', url)
    
    # Dividir la URL procesada en tokens (palabras)
    tokens = url.split()
    
    return tokens

tokens_list = df["url"].apply(extract_tokens)

all_tokens = [token for tokens in tokens_list for token in tokens]

token_freq = Counter(all_tokens)

# Crear la wordcloud a partir de los tokens y sus frecuencias
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(token_freq)

# Mostrar la wordcloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir la lista de tokens en una lista de strings
tokens_list_str = [" ".join(tokens) for tokens in tokens_list]

# Crear un objeto TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, max_df=0.8, min_df=5)

# Aplicar TfidfVectorizer a la lista de tokens
X = vectorizer.fit_transform(tokens_list_str).toarray()

y = df["is_spam"]

# Mostrar las primeras 5 filas de la matriz X
X[:5]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define los hiperparámetros a ajustar
param_grid = {
    'C': [0.1, 1, 10, 100],                    # Parámetro de regularización
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Tipo de kernel
    'gamma': ['scale', 'auto'],                # Coeficiente del kernel
    'class_weight': [None, 'balanced'],        # Peso de las clases
    'probability': [True, False],               # Habilitar predicción de probabilidades
    'shrinking': [True, False],                 # Reducción de vectores de soporte
}

# Instancia el modelo SVC
svm_model = SVC()

# Realiza la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')

# Ajusta el modelo a los datos
grid_search.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Muestra la mejor puntuación obtenida
print("Mejor puntuación:", grid_search.best_score_)

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)
y_pred

In [ ]:
y_pred.shape

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)*100

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calcula las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # o 'micro', 'macro', 'weighted'
recall = recall_score(y_test, y_pred, average='weighted')  # o 'micro', 'macro', 'weighted'
f1 = f1_score(y_test, y_pred, average='weighted')  # o 'micro', 'macro', 'weighted'

# Imprime las métricas
print("Accuracy:", accuracy*100)
print("Precision:", precision*100)
print("Recall:", recall*100)
print("F1 Score:", f1*100)

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_iris
import joblib

# Entrenar un modelo SVM de ejemplo (puedes reemplazar esto con tu propio modelo)
X, y = load_iris(return_X_y=True)
model = SVC(kernel='linear')
model.fit(X, y)

# Guardar el modelo en un archivo
joblib.dump(model, 'modelo_svm.pkl')